In [ ]:
import pymysql
import pandas as pd
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
scale = MinMaxScaler()


在实验练习08中我们使用的数据为两个城市在不同因素影响下某小时共享单车租用的数量。现在我们利用该数据集构建一个回归预测模型，根据数据中的某些属性预测该时段内的单车租用数量，该表所有值均为整型数据，表结构为：

id 记录编号，无其他意义
city 城市代号，0为北京，1为上海
hour 小时，代表时间
is_workday 是否为工作日，0为否，1为是
temp_air 大气温度，单位为摄氏度
temp_body 体感温度，单位为摄氏度
weather 天气代号，1为晴天，2为多云或阴天，3为雨天或雪天
wind 风级，数值越大代表风速越大
y 该小时内共享单车被租用的数量

In [ ]:
db = pymysql.connect(host="cdb-r2g8flnu.bj.tencentcdb.com", port=10209,
                     user="dase2020", password="dase2020", database="dase_intro_2020")
cursor = db.cursor()  # 使用 cursor() 方法创建一个游标对象 cursor,执行SQL语句都是通过游标对象实现


sql = "SELECT * FROM bicycle_train"
cursor.execute(sql)
result = cursor.fetchall()

数据获取：数据集已下载至bike.csv文件中，请使用pandas库读取该文件。
提示：原始数据集共10,000条数据。

In [ ]:
lst = []
for i in result:
    lst.append(list(i))


header = ["id", "city", "hour", "is_workday", "weather", "temp_1", "temp_2", "wind", "y"]


with open(r"E:\学习\数据科学与工程导论\lab11\bike.csv", "w", encoding='utf-8', newline="") as f1:
    writer = csv.writer(f1)
    writer.writerow(header)
    for i in lst:
        writer.writerow(i)


data = pd.read_csv(r"E:\学习\数据科学与工程导论\lab11\bike.csv")

数据预处理 I：id属性对构建回归预测模型没有帮助，请剔除掉该列。

In [ ]:
data_1 = data.drop('id', axis=1)

数据预处理 II：我们暂不考虑不同城市对单车租用的影响，请筛选出上海市的所有数据，然后剔除city列。
提示：目前剩余4,998条数据。

In [ ]:
data_2 = data_1.loc[data_1['city'] == 1]
data_2 = data_2.drop('city', axis=1)

数据预处理 III：为简化数据，请将hour列中原来6点-18点统一为1；19点-次日5点统一为0。

In [ ]:
data_2.loc[data_2['hour'] >= 19, 'hour'] = 0
data_2.loc[data_2['hour'] <= 5, 'hour'] = 0
data_2.loc[data_2['hour'] != 0, 'hour'] = 1

数据预处理 IV：y列为单车租用数量，是我们的预测目标（标签），请将该列提取出来，并转换为一个numpy列向量，将原先的y列剔除。

In [ ]:
y = data_2['y'].to_numpy()
y = y.reshape(len(y), 1)
data_2 = data_2.drop('y', axis=1)

数据预处理 V：请将DataFrame对象转换为Numpy数组，方便后续操作。

In [ ]:
data_2 = data_2.to_numpy()

数据集划分：请按照训练集与测试集8:2的比例将原始数据集划分。

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_2, y, test_size=0.2)

数据预处理 VI：请分别对训练集数据、训练集标签、测试集数据和测试集标签进行归一化。

In [ ]:
x_train = scale.fit_transform(x_train)
x_test = scale.fit_transform(x_test)
y_train = scale.fit_transform(y_train)
y_test = scale.fit_transform(y_test)

模型构建：请先构建一个线性回归模型（多元一次函数），然后利用训练集训练模型。

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

模型测试：利用测试集对训练好的模型进行评估。
提示：使用predict(data_array)方法输入测试集，该函数返回值为模型预测值。

In [ ]:
y_predict = model.predict(x_test)

模型评估：请使用均方根误差（RMSE）作为评估指标，并输出RMSE值。
均方根误差(Root of Mean Squared Error)，公式为


In [ ]:
print(mean_squared_error(y_test, y_predict)**0.5)